In [10]:
# desc

In [11]:
import json
import uuid

import requests

# import the smartpynector module
from NBs import smartpynector as sp


In [12]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
THERMOSTAT_API_URL = "http://127.0.0.1:8000/thermostat"


In [13]:
# Defining the Smart Connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=THERMOSTAT_API_URL,
    knowledgeBaseName="thermostat 1",
    knowledgeBaseDescription="thermostat 1",
    reasonerEnabled=False,
)

In [14]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-12 16:19:24 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:26 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-12 16:19:26 INFO Request successful
2023-05-12 16:19:26 DEBUG 


<Response [200]>

In [15]:

GRAPH_PATTERN = '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
}
HEADERS = {
    "Content-Type": "application/json",
    "knowledge-Base-Id": THERMOSTAT_API_URL,
}


In [16]:
answer_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AnswerKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers=HEADERS,
).json()["knowledgeInteractionId"]

answer_ki_id

2023-05-12 16:19:27 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:27 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-12 16:19:27 INFO Request successful
2023-05-12 16:19:27 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/be4fdc09-a3a1-46d5-b6ad-e802510e167d"}


'http://127.0.0.1:8000/thermostat/interaction/be4fdc09-a3a1-46d5-b6ad-e802510e167d'

In [17]:
thermostat_actual_temperature = requests.get(THERMOSTAT_API_URL + '/actual_temperature').json()
thermostat_actual_temperature

2023-05-12 16:19:27 DEBUG Starting new HTTP connection (1): 127.0.0.1:8000
2023-05-12 16:19:27 DEBUG http://127.0.0.1:8000 "GET /thermostat/actual_temperature HTTP/1.1" 200 2


20

In [18]:
# %%timeit
sp.perform_answer_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=THERMOSTAT_API_URL,
    knowledge_interaction_id=answer_ki_id,
    answer_binding_set=[{
        "meas": f'<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>',
        "temp": f"{thermostat_actual_temperature}",
        "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
        "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
        "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
    }]
)

2023-05-12 16:19:27 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:30 DEBUG http://127.0.0.1:8280 "GET /rest/sc/handle HTTP/1.1" 200 209
2023-05-12 16:19:30 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:30 DEBUG http://127.0.0.1:8280 "POST /rest/sc/handle HTTP/1.1" 200 0


<Response [200]>

In [19]:
# Lets try implementing a POST / REACT knowledge interaction

In [20]:
react_ki = sp.register_knowledge_interaction(
    'ReactKnowledgeInteraction', KNOWLEDGE_ENGINE_URL, PREFIXES, headers= HEADERS,
    argument_graph_pattern="?a rdf:type saref:Measurement . ?a saref:hasValueInMeters ?b .",
    result_graph_pattern="?a rdf:type saref:Measurement . ?a saref:hasValueInYards ?b .",
)
react_ki.json()['knowledgeInteractionId']

2023-05-12 16:19:31 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:31 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-12 16:19:31 INFO Request successful
2023-05-12 16:19:31 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/e1ad3a09-361b-4044-aa45-21306f414abc"}


'http://127.0.0.1:8000/thermostat/interaction/e1ad3a09-361b-4044-aa45-21306f414abc'